In [1]:
import os
import datetime
import numpy as np
import pandas as pd

import QuantExt as qe

In [2]:
from quant_lib.fx_swap_curve import get_quote, usdirs_curve, krwccs_curve

In [3]:
class FXF():
    def __init__(self, today, maturity_date, fx_spot, fx_forward, usd_notional, position):
        
        # initial setup
        self.date = today
        self.usd_curve = self.usd_curve(self.date)
        self.krw_curve = self.krw_curve(self.date)
        self.fx_spot = fx_spot


        self.maturity_date = qe.Date(maturity_date.day, maturity_date.month, maturity_date.year)
        self.usd = qe.KRWCurrency()
        self.krw = qe.USDCurrency()
        self.usd_notional = usd_notional
        self.krw_notional = usd_notional * fx_forward

        self.day_count = qe.ActualActual()

        if position == 'long':
            self.payCcy1 = True
        else:
            self.payCcy1 = False


        # pricing result
        self.npv = self.PRICING(self.usd_curve, self.krw_curve, self.fx_spot)
        self.fx_delta = self.FX_DELTA()
        self.usd_ir_delta = self.USD_IR_DELTA()
        self.krw_ir_delta = self.KRW_IR_DELTA()
        self.theta = self.THETA()
    
    def usd_curve(self, date):
        return usdirs_curve(date, get_quote(date, 'USD'))
    
    def krw_curve(self, date):
        return krwccs_curve(date, get_quote(date, 'KRW'))


    def PRICING(self, usd_curve, krw_curve, fx_spot):
        
        # Handles of Market variables
        usd_curve_handle = qe.YieldTermStructureHandle(usd_curve)
        krw_curve_handle = qe.YieldTermStructureHandle(krw_curve)
        fx_spot_handle = qe.QuoteHandle(qe.SimpleQuote(fx_spot))

        # FX forward instrument
        fxf = qe.FxForward(self.krw_notional,
                           self.krw,
                           self.usd_notional,
                           self.usd,
                           self.maturity_date,
                           self.payCcy1 
                        )

        engine = qe.DiscountingFxForwardEngine(self.krw,
                                               krw_curve_handle,
                                               self.usd,
                                               usd_curve_handle,
                                               fx_spot_handle
                                            )

        # conduct pricing
        fxf.setPricingEngine(engine)

        # fx forward price
        npv = fxf.NPV()

        return npv

    
    def FX_DELTA(self):

        percentage = 0.01

        # FX forward price when 1% up
        up_fx = self.fx_spot * (1 + percentage)
        up_fxf = self.PRICING(self.usd_curve, self.krw_curve, up_fx)

        # FX forward price when 1% down
        down_fx = self.fx_spot * (1 - percentage)
        down_fxf = self.PRICING(self.usd_curve, self.krw_curve, down_fxf)

        return  (up_fxf - down_fxf) / 2




    def USD_IR_DELTA(self):
        # Handle of USD curve
        curve_handle = qe.YieldTermStructureHandle(self.usd_curve)

        # 1 bp
        basis_point = 0.0001

        # FX forward price when 1bp up
        up_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        up_fxf = self.PRICING(up_curve, self.krw_curve, self.fx_spot)

        # FX forward price when 1bp down
        down_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(-basis_point)))
        down_fxf = self.PRICING(down_curve, self.krw_curve, self.fx_spot)       

        return (up_fxf - down_fxf) / 2
    
    
    def KRW_IR_DELTA(self):
        # Handle of KRW curve
        curve_handle = qe.YieldTermStructureHandle(self.krw_curve)

        # 1 bp
        basis_point = 0.0001

        # FX forward price when 1bp up
        up_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        up_fxf = self.PRICING(self.usd_curve, up_curve, self.fx_spot)

        # FX forward price when 1bp down
        down_curve = qe.ZeroSpreadedTermStructure(curve_handle, qe.QuoteHandle(qe.SimpleQuote(basis_point)))
        down_fxf = self.PRICING(self.usd_curve, down_curve, self.fx_spot)

        return (up_fxf - down_fxf) / 2



    def THETA(self):
        # theta is change in value if one unit time passes.
        # in here, unit time is 1 day
        # since derivative product have time value, time to maturity is major variable in pricing derivatives
        price_t0 = self.PRICING(self.usd_curve, self.krw_curve, self.fx_spot)

        # FX Forward price at t1
        usd_curve_t1 = self.usd_curve(self.date + datetime.datetime(days=1))
        krw_curve_t1 = self.krw_curve(self.date + datetime.datetime(days=1))

        price_t1 = self.PRICING(usd_curve_t1, krw_curve_t1, self.fx_spot)

        return price_t1 - price_t0

In [4]:
todays_date = datetime.date(2020, 10, 9)
maturity_date = datetime.date(2021, 10, 9)

In [5]:
position = 'long'
fx_spot = 1153.30
fx_forward = 1152.32

usd_notional = 10000000
krw_notional = usd_notional * fx_forward

In [ ]:
# build FXF object
fxf = FXF(today=todays_date,
        maturity_date=maturity_date,
        fx_spot=fx_spot,
        fx_forward=fx_forward,
        usd_notional=usd_notional,
        position=position
)

In [3]:
print("price of FXF = {}".format(round(fxf.npv,4)))
print("FX Delta = {}".format(round(fxf.fx_delta,4)))
print("USD IR Delta = {}".format(round(fxf.usd_ir_delta,4)))
print("KRW IR Delta = {}".format(round(fxf.krw_ir_delta,4)))
print("Theta = {}".format(round(fxf.theta,4)))

price of FXF = -25312.9725
FX Delta = 115055468.771
USD IR Delta = -1166534.6159
KRW IR Delta = 1150557.2209
Theta = -48267.3058
